## 1. Open Data

Open data is the idea that some data should be freely available to everyone to use and republish as they wish, without restrictions from copyright, patents or other mechanisms of control. The goals of the open data movement are similar to those of other "open" movements such as [open source](https://en.wikipedia.org/wiki/Open-source_software_movement), [open hardware](https://en.wikipedia.org/wiki/Open_hardware), [open content](https://en.wikipedia.org/wiki/Open_content), [open government](https://en.wikipedia.org/wiki/Open_government) and [open access](https://en.wikipedia.org/wiki/Open_access_(publishing). The philosophy behind open data has been long established, but the term "open data" itself is recent, gaining popularity with the rise of the Internet and World Wide Web and, especially, with the launch of open-data government initiatives such as [Data.gov](https://en.wikipedia.org/wiki/Data.gov) and [Data.gov.uk](https://en.wikipedia.org/wiki/Data.gov.uk).

In this mission we will explore the open data initiative of [Natal-RN](http://ckan.imd.ufrn.br/). The idea is to analyze the correlation between geographic and education data. 

<img width="600" alt="creating a repo" src="https://drive.google.com/uc?export=view&id=0BxhVm1REqwr0WmkzdlY2NDdMZ1k">


## 2. Municipal schools 

The dataset provides information about public elementary schools and kindergartens of Natal-RN in 2014.

source: http://ckan.imd.ufrn.br/dataset/quadro-das-escolas-e-cmeis-do-municipio


## 3. Import essential packages

In [152]:
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import folium
from folium.plugins import HeatMap
import branca
from folium import plugins

Install geocoder 

>```python
conda install -c conda-forge geocoder
```

In [76]:
import geocoder

## 4. Collect the data

In [127]:
# URL - see ckan.imd.ufrn.br for further information
url_school = 'http://ckan.imd.ufrn.br/dataset/9b362c15-832b-4aa9-9dfe-a5e015b3ce54/resource/\
99e0eef6-e16c-4ed8-bf62-d6bca9626eeb/download/escolas-por-regioes-administrativas.csv'

url_kgarten = 'http://ckan.imd.ufrn.br/dataset/9b362c15-832b-4aa9-9dfe-a5e015b3ce54/resource/\
6d8e8580-bb48-4d75-a55c-4fa04da23919/download/cmeis-por-regioes-administrativas.csv'

# Read the dataset
df_school = pd.read_csv(url_school, encoding = 'utf-8', sep = ';')
df_kgarten = pd.read_csv(url_kgarten, encoding = 'utf-8', sep = ';')

## 5. Exploratory data analysis (EDA)

### 5.1 Elementary school

In [78]:
# print basic information about the elementary schools
print(df_school.columns)
print(df_school.shape)
print(df_school.info())
df_school.head()

Index(['Região Administrativa', 'CÓDIGO', 'ESTABELECIMENTO', 'ENDEREÇO', 'Nº',
       'BAIRRO', 'CEP', 'FONE'],
      dtype='object')
(72, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 8 columns):
Região Administrativa    72 non-null object
CÓDIGO                   72 non-null int64
ESTABELECIMENTO          72 non-null object
ENDEREÇO                 72 non-null object
Nº                       68 non-null object
BAIRRO                   72 non-null object
CEP                      72 non-null int64
FONE                     72 non-null int64
dtypes: int64(3), object(5)
memory usage: 4.6+ KB
None


,Região Administrativa,CÓDIGO,ESTABELECIMENTO,ENDEREÇO,Nº,BAIRRO,CEP,FONE
0,SUL,24058890,ESC MUL PROF ANTÔNIO SEVERIANO,AV OURO PRETO,2754,NEÓPOLIS,59088690,32324762
1,SUL,24058912,ESC MUL PROF ARNALDO MONTEIRO BEZERRA,ARACITABA,2993,NEÓPOLIS,59084080,32324763
2,SUL,24060690,ESC MUL PROF ASCENDINO DE ALMEIDA,RUA JOAQUIM CARDOSO,NaN,PITIMBU,59069010,32324767
3,SUL,24058793,ESC MUL PROF CARLOS BELLO MORENO,RUA ARAPIRACA,SN,NEÓPOLIS,59086210,32324761
4,SUL,24075710,ESC MUL PROF OTTO DE BRITO GUERRA,RUA SERRA DA JUREMA,SN,PITIMBU,59068150,32328373


### 5.2 Kindergarten

In [129]:
# print basic information about the kindergarten
print(df_kgarten.columns)
print(df_kgarten.shape)
print(df_kgarten.info())
df_kgarten.head()

Index(['Região Administrativa', 'CÓDIGO', 'ESTABELECIMENTO', 'ENDEREÇO', 'Nº',
       'BAIRRO', 'CEP', 'FONE'],
      dtype='object')
(73, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 8 columns):
Região Administrativa    73 non-null object
CÓDIGO                   73 non-null int64
ESTABELECIMENTO          73 non-null object
ENDEREÇO                 72 non-null object
Nº                       72 non-null object
BAIRRO                   73 non-null object
CEP                      73 non-null int64
FONE                     67 non-null float64
dtypes: float64(1), int64(2), object(5)
memory usage: 4.6+ KB
None


,Região Administrativa,CÓDIGO,ESTABELECIMENTO,ENDEREÇO,Nº,BAIRRO,CEP,FONE
0,SUL,24077720,CMEI CLAUDETE COSTA MACIEL,RUA SERRA DOS CARAJAS,3160,PITIMBU,59068200,32328403.0
1,SUL,24077739,CMEI HAYDEE MONTEIRO BEZERRA DE MELO,RUA JOSÉ SELEDON,70,PONTA NEGRA,59090215,32328413.0
2,SUL,24056936,CMEI KÁTIA FAGUNDES GARCIA,RUA PROFESSORA ANA DJANIRA,1960,CANDELÁRIA,59064480,87291989.0
3,SUL,24077267,CMEI MARIA CELONI CAMPOS,RUA BAIA FORMOSÁ,1517,LAGOA NOVA II,59063060,32329443.0
4,SUL,24077500,CMEI MOEMA TINOCO DA CUNHA LIMA,RUA JACUI,217,NEÓPOLIS,59080270,32328376.0


### 5.3 Grouping the data by region

In [130]:
# Schools
schools_by_region = pd.DataFrame(df_school.groupby(['Região Administrativa'])['CÓDIGO'].count()).reset_index()
schools_by_region.rename(columns = {
    'Região Administrativa': 'Region',
    'CÓDIGO': 'Amount'
}, inplace = True)

# Kindergarten
kgarten_by_region = pd.DataFrame(df_kgarten.groupby(['Região Administrativa'])['CÓDIGO'].count()).reset_index()
kgarten_by_region.rename(columns = {
    'Região Administrativa': 'Region',
    'CÓDIGO': 'Amount'
}, inplace = True)

In [131]:
schools_by_region.head()

,Region,Amount
0,LESTE,9
1,NORTE,34
2,OESTE,21
3,SUL,8


In [132]:
kgarten_by_region.head()

,Region,Amount
0,LESTE,10
1,NORTE,27
2,OESTE,24
3,SUL,12


## 6. Cleaning and preparing the data

### 6.1 Normalize the missing data

In [133]:
# Normalize NaN values on 'Nº' column
df_school.loc[:, 'Nº'].fillna('', inplace=True)

# Normalize the unidentified address
df_school.loc[df_school[df_school['Nº'] == 'SN'].index,'Nº'] = ''
df_kgarten.loc[df_kgarten[df_kgarten['Nº'] == 'SN'].index,'Nº'] = ''

# Normalize NaN values on 'ENDEREÇO' and 'FONE' column
df_kgarten.loc[:, 'ENDEREÇO'].fillna('', inplace=True)
df_kgarten.loc[:, 'FONE'].fillna('', inplace=True)

In [134]:
df_school.info()
df_kgarten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 8 columns):
Região Administrativa    72 non-null object
CÓDIGO                   72 non-null int64
ESTABELECIMENTO          72 non-null object
ENDEREÇO                 72 non-null object
Nº                       72 non-null object
BAIRRO                   72 non-null object
CEP                      72 non-null int64
FONE                     72 non-null int64
dtypes: int64(3), object(5)
memory usage: 4.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 8 columns):
Região Administrativa    73 non-null object
CÓDIGO                   73 non-null int64
ESTABELECIMENTO          73 non-null object
ENDEREÇO                 73 non-null object
Nº                       72 non-null object
BAIRRO                   73 non-null object
CEP                      73 non-null int64
FONE                     73 non-null object
dtypes: int64(2), object(6)
memory usage: 4.6+ 

### 6.2 Preparing  geocoder information for education units

In [135]:
# FOR ELEMENTARY SCHOOLS

# convert the colum 'CEP' to str
df_school['CEP'] = df_school['CEP'].astype(str)

# Address
df_school['GEOCODE_INPUT'] = df_school['ENDEREÇO'] + ', ' + df_school['Nº'] + ', NATAL-RN'

# Initialize Lat and Lng
df_school['LAT'], df_school['LNG'] = [0, 0]

# Type of education unit
df_school['Type'] = 'Elem'

# New dataset 
schools_geolocation = df_school[['ESTABELECIMENTO', 'Type', 'GEOCODE_INPUT', 'LAT', 'LNG']]
schools_geolocation = schools_geolocation.rename(columns = {'ESTABELECIMENTO':'NAME'})

# See the modification
schools_geolocation.head()

,NAME,Type,GEOCODE_INPUT,LAT,LNG
0,ESC MUL PROF ANTÔNIO SEVERIANO,Elem,"AV OURO PRETO, 2754, NATAL-RN",0,0
1,ESC MUL PROF ARNALDO MONTEIRO BEZERRA,Elem,"ARACITABA, 2993, NATAL-RN",0,0
2,ESC MUL PROF ASCENDINO DE ALMEIDA,Elem,"RUA JOAQUIM CARDOSO, , NATAL-RN",0,0
3,ESC MUL PROF CARLOS BELLO MORENO,Elem,"RUA ARAPIRACA, , NATAL-RN",0,0
4,ESC MUL PROF OTTO DE BRITO GUERRA,Elem,"RUA SERRA DA JUREMA, , NATAL-RN",0,0


In [136]:
# FOR KINDERGARTENS

# convert the colum 'CEP' to str
df_kgarten['CEP'] = df_kgarten['CEP'].astype(str)

# Address
df_kgarten['GEOCODE_INPUT'] = df_kgarten['ENDEREÇO'] + ', ' + df_kgarten['Nº'] + ', NATAL-RN'

# Initialize Lat and Lng
df_kgarten['LAT'], df_kgarten['LNG'] = [0, 0]

# Type of education unit
df_kgarten['Type'] = 'KGAR'

# New dataset 
kgarten_geolocation = df_kgarten[['ESTABELECIMENTO', 'Type', 'GEOCODE_INPUT', 'LAT', 'LNG']]
kgarten_geolocation = kgarten_geolocation.rename(columns = {'ESTABELECIMENTO':'NAME'})

# See the modification
kgarten_geolocation.head()

,NAME,Type,GEOCODE_INPUT,LAT,LNG
0,CMEI CLAUDETE COSTA MACIEL,KGAR,"RUA SERRA DOS CARAJAS, 3160, NATAL-RN",0,0
1,CMEI HAYDEE MONTEIRO BEZERRA DE MELO,KGAR,"RUA JOSÉ SELEDON, 70, NATAL-RN",0,0
2,CMEI KÁTIA FAGUNDES GARCIA,KGAR,"RUA PROFESSORA ANA DJANIRA, 1960, NATAL-RN",0,0
3,CMEI MARIA CELONI CAMPOS,KGAR,"RUA BAIA FORMOSÁ, 1517, NATAL-RN",0,0
4,CMEI MOEMA TINOCO DA CUNHA LIMA,KGAR,"RUA JACUI, 217, NATAL-RN",0,0


### 6.3 Concatenate the datasets

In [137]:
# Concatenate the dataframe
frames = [schools_geolocation, kgarten_geolocation]
df_all_geo = pd.concat(frames, ignore_index = True) 
df_all_geo.head()

,NAME,Type,GEOCODE_INPUT,LAT,LNG
0,ESC MUL PROF ANTÔNIO SEVERIANO,Elem,"AV OURO PRETO, 2754, NATAL-RN",0,0
1,ESC MUL PROF ARNALDO MONTEIRO BEZERRA,Elem,"ARACITABA, 2993, NATAL-RN",0,0
2,ESC MUL PROF ASCENDINO DE ALMEIDA,Elem,"RUA JOAQUIM CARDOSO, , NATAL-RN",0,0
3,ESC MUL PROF CARLOS BELLO MORENO,Elem,"RUA ARAPIRACA, , NATAL-RN",0,0
4,ESC MUL PROF OTTO DE BRITO GUERRA,Elem,"RUA SERRA DA JUREMA, , NATAL-RN",0,0


### 6.4 Calculate the geolocalization from Geocoder

The Geocoder API can block your request if multiples queries using the same IP address are sent simultaneously. Thus we recommend load the dataset available in repository (geo.csv).

In [138]:
# Use this cell to speed up the calc of geolocalization
# Case affirmative, jump to section 7
df_all_geo = pd.read_csv('edunatal.csv')
df_all_geo.head()

,Unnamed: 0,NAME,Type,GEOCODE_INPUT,LAT,LNG
0,0,ESC MUL PROF ANTÔNIO SEVERIANO,Elem,"AV OURO PRETO, 2754, NATAL-RN",-5.870580,-35.214868
1,1,ESC MUL PROF ARNALDO MONTEIRO BEZERRA,Elem,"ARACITABA, 2993, NATAL-RN",-5.868884,-35.197566
2,2,ESC MUL PROF ASCENDINO DE ALMEIDA,Elem,"RUA JOAQUIM CARDOSO, , NATAL-RN",-5.851115,-35.240676
3,3,ESC MUL PROF CARLOS BELLO MORENO,Elem,"RUA ARAPIRACA, , NATAL-RN",-5.863614,-35.207138
4,4,ESC MUL PROF OTTO DE BRITO GUERRA,Elem,"RUA SERRA DA JUREMA, , NATAL-RN",-5.855495,-35.245909


In [97]:
# Cleaning some row with problems (accent and strange character)

df_all_geo.loc[29,'GEOCODE_INPUT'] = 'MIGUEL CASTRO,NATAL-RN'
df_all_geo.loc[73,'GEOCODE_INPUT']  = 'RUA JOSE SELEDON, 70, NATAL-RN'
df_all_geo.loc[106,'GEOCODE_INPUT']  = 'Rua Alfredo Edeltrudes, 2300, NATAL-RN'

In [98]:
# Calculate the geolocation info 
for i in tqdm(range(len(df_all_geo))):
    g = geocoder.google(df_all_geo.loc[i,'GEOCODE_INPUT'])
    df_all_geo.loc[i,'LAT'] = g.lat
    df_all_geo.loc[i,'LNG'] = g.lng

100%|██████████| 145/145 [02:41<00:00,  1.13s/it]


In [99]:
# Finally, the dataset is ready for the next step (analyzing)
df_all_geo.head()

,NAME,Type,GEOCODE_INPUT,LAT,LNG
0,ESC MUL PROF ANTÔNIO SEVERIANO,Elem,"AV OURO PRETO, 2754, NATAL-RN",-5.870580,-35.214868
1,ESC MUL PROF ARNALDO MONTEIRO BEZERRA,Elem,"ARACITABA, 2993, NATAL-RN",-5.868884,-35.197566
2,ESC MUL PROF ASCENDINO DE ALMEIDA,Elem,"RUA JOAQUIM CARDOSO, , NATAL-RN",-5.851115,-35.240676
3,ESC MUL PROF CARLOS BELLO MORENO,Elem,"RUA ARAPIRACA, , NATAL-RN",-5.863614,-35.207138
4,ESC MUL PROF OTTO DE BRITO GUERRA,Elem,"RUA SERRA DA JUREMA, , NATAL-RN",-5.855495,-35.245909


In [102]:
# Save the dataset to a csv file
df_all_geo.to_csv('edunatal.csv')

## 7. Data Analysis 

In [150]:
# Create a map object
m = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain'
)

# Dictionary for colors
unit_type_colors = {
    'Elem': 'green',
    'KGAR': 'red',
}

# Dictionary for icons
unit_type_icons = {
    'Elem': 'graduation-cap',
    'KGAR': 'odnoklassniki',
}

In [154]:
# Ploting the educational units on map
for i in tqdm(range(len(df_all_geo))):
    folium.Marker([df_all_geo.loc[i,'LAT'], df_all_geo.loc[i,'LNG']],
                  icon=folium.Icon(
                          color = unit_type_colors[df_all_geo.loc[i,'Type']],
                          icon = unit_type_icons[df_all_geo.loc[i,'Type']],
                          prefix='fa'),
                  popup = df_all_geo.loc[i, 'NAME']
        ).add_to(m)

# Add fullscreen button
plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True).add_to(m)

m

100%|██████████| 145/145 [00:04<00:00, 34.90it/s]


In [140]:
coordinates_elem = []
coordinates_kgarten = []

for i in tqdm(range(len(df_all_geo))):
    if (df_all_geo.loc[i,'Type'] == 'Elem'):
        coordinates_elem.append([df_all_geo.loc[i,'LAT'], df_all_geo.loc[i,'LNG'], 1])
    else:
        coordinates_kgarten.append([df_all_geo.loc[i,'LAT'], df_all_geo.loc[i,'LNG'], 1])
        
# Create a map object
m_elem = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain'
)

# Create a map object
m_kgarten = folium.Map(
    location=[-5.826592, -35.212558],
    zoom_start=12,
    tiles='Stamen Terrain'
)

# Create a heat map
HeatMap(coordinates_elem).add_to(m_elem)
HeatMap(coordinates_kgarten).add_to(m_kgarten)


100%|██████████| 145/145 [00:00<00:00, 28509.94it/s]


In [155]:
f = branca.element.Figure()

d1 = f.add_subplot(1, 2, 1)
d2 = f.add_subplot(1, 2, 2)

d1.add_child(m_elem)
d2.add_child(m_kgarten)

#m_elem

<br>
<div class="alert alert-info">
<b>Exercise Start.</b>
</div>

**Description**:

1. Create a cluster using the localization data for educational units of Natal-RN.
2. Generate heatmap figures of elementary and kindergarten schools in Natal-RN, assuming the number of teachers in each educational unit as weight of heatmap. 

In [143]:
# Tips
url_prof_school = 'http://ckan.imd.ufrn.br/dataset/0cc7f31d-1fe7-4232-82fd-0ef356d62342/resource/c06090bb-506c-4193-a115-02840d6635ea/download/funcao-docente-do-ens.-fundamental-por-estabelecimento.csv'
url_prof_kgarten = 'http://ckan.imd.ufrn.br/dataset/0cc7f31d-1fe7-4232-82fd-0ef356d62342/resource/06356b96-82c3-4969-a072-63029dd76a97/download/funcao-docente-do-ens.-infantil-por-estabelecimento.csv'

df_prof_school =  pd.read_csv(url_prof_school, encoding = 'utf-8', sep = ';')
df_prof_kgarten = pd.read_csv(url_prof_kgarten, encoding = 'utf-8', sep = ';')

In [144]:
df_prof_school.head()

,Estabelecimento,Ens. Fundamental,Ens. Médio Magistério,Ens. Médio,Ens. Superio,Especialização,Mestrado,Doutorado,Nenhum
0,ESC MUL 4º CENTENÁRIO,0,0,0,28,12,2,0,14
1,ESC MUL CELESTINO PIMENTEL,0,0,0,34,24,1,0,10
2,ESC MUL CHICO SANTEIRO,0,0,0,16,6,0,0,10
3,ESC MUL DJALMA MARANHÃO,0,0,0,24,12,2,0,12
4,ESC MUL ESTUDANTE EMMANUEL BEZERRA,0,0,0,45,32,2,0,13


In [145]:
df_prof_kgarten.head()

,Estabelecimento,Ens. Fundamental,Ens. Médio Magistério,Ens. Médio,Ens. Superio,Especialização,Mestrado,Doutorado,Nenhum
0,CMEI AMOR DE MÃE,0,0,0,27,1,0,0,26
1,CMEI BELCHIOR JORGE DE SÁ,0,0,0,13,1,0,0,12
2,CMEI BOM SAMARITANO,0,0,0,6,1,0,0,5
3,CMEI CARMEM FERNANDES PEDROZA,0,0,0,14,5,0,0,9
4,CMEI CLARA CAMARÃO,0,0,0,11,1,0,0,10


In [146]:
df_prof_school['Total'] = df_prof_school.sum(axis=1)
df_prof_school.head()

,Estabelecimento,Ens. Fundamental,Ens. Médio Magistério,Ens. Médio,Ens. Superio,Especialização,Mestrado,Doutorado,Nenhum,Total
0,ESC MUL 4º CENTENÁRIO,0,0,0,28,12,2,0,14,56
1,ESC MUL CELESTINO PIMENTEL,0,0,0,34,24,1,0,10,69
2,ESC MUL CHICO SANTEIRO,0,0,0,16,6,0,0,10,32
3,ESC MUL DJALMA MARANHÃO,0,0,0,24,12,2,0,12,50
4,ESC MUL ESTUDANTE EMMANUEL BEZERRA,0,0,0,45,32,2,0,13,92


In [147]:
df_prof_kgarten['Total'] = df_prof_kgarten.sum(axis=1)
df_prof_kgarten.head()

,Estabelecimento,Ens. Fundamental,Ens. Médio Magistério,Ens. Médio,Ens. Superio,Especialização,Mestrado,Doutorado,Nenhum,Total
0,CMEI AMOR DE MÃE,0,0,0,27,1,0,0,26,54
1,CMEI BELCHIOR JORGE DE SÁ,0,0,0,13,1,0,0,12,26
2,CMEI BOM SAMARITANO,0,0,0,6,1,0,0,5,12
3,CMEI CARMEM FERNANDES PEDROZA,0,0,0,14,5,0,0,9,28
4,CMEI CLARA CAMARÃO,0,0,0,11,1,0,0,10,22
